# 검증세트
- 테스트 세트를 반복적으로 사용하여 성능 검증 -> 결과를 테스트 세트에 맞추게 됨
- 테스트 세트는 가능한 마지막에 사용하는 것이 적절
=> 테스트 세트를 사용하지 않고 훈련 세트를 나누어 측정

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [5]:
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [6]:
# train_input과 train_target을 다시 train_test_split() 함수에 넣어 훈현 세트와 검증 세트 만들어줌
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [7]:
# 전체 데이터 6497 중 
# 훈련 세트 : 4157, 검증 세트 : 1040, 테스트 세트 : 1300
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
# 과대적합

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증(Cross Validation)
- 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터 사용 가능
- 검증 세트를 떼어내어 평가하는 과정 여러번 반복
- 점수를 평균하여 최종 검증 점수를 얻음
- 5-폴드 교차검증, 10-폴드 교차 검증 자주 사용

In [9]:
# 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate 함수에 전달
# cross_validate(): test_score만 반환

from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00790238, 0.00715899, 0.00761485, 0.007303  , 0.00710797]), 'score_time': array([0.00090623, 0.00070906, 0.00068784, 0.00072861, 0.00070381]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [10]:
# test_score이지만 검정 폴드의 점수

import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


### 분할기
- 교차 검증에서 폴드를 어떻게 나눌지 결정
- 회귀 : KFold
- 분류 : StratifiedKFold

In [11]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [15]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증 수행

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터
- 파라미터 : 머신러닝 모델이 학습하는 파라미터
- 하이퍼 파라미터 : 사용자가 지정
  - 클래스나 메서드의 매개변수로 표현

# 그리드 서치
- 그리드 서치를 사용하여 최적의 하이퍼 파라미터를 찾아감
- 하이퍼파라미터의 탐색과 교차 검증을 한 번에 수행
- 최적의 하이퍼 파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 함  <br>
  -> 그리드 서치는 최적의 파라미터 조합으로 자동으로 다시 모델 훈련

In [16]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [17]:
# 결정 트리 객체를 생성하자마자 바로 전달
# 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [18]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [20]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [21]:
# 최적의 파라미터는 best_estimator_에 저장되어있음
print(gs.best_estimator_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)


In [22]:
# 수행한 교차검증의 평균 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [23]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 탐색할 매개변수 지정
2. 그리드 서치 수행 -> 최상의 평균 검증 점수가 나오는 매개변수 조합 찾음 
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용하여 최종 모델 훈련

In [24]:
# 수행할 교차 검증 횟수 : 9*15*10=1,350
# 기본 5 폴드 교차 검증 수행 -> 모델의 수 6,750개
{'min_impurity_decrease': 0.0001}
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [25]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [26]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [28]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 랜덤 서치
- 매개변수의 값이 수치일때 범위나 간격을 미리 정하기 어려울 수 있음
- 너무 많은 매개 변수 조건이 있어 그리드 서치 수행 기간이 오래 걸릴 수 있음<br>
=> 랜덤 서치 사용
- 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달


In [29]:
from scipy.stats import uniform, randint

In [30]:
# 모든 주어진 범위에서 고르게 값을 뽑음
# randint : 정숫값

rgen = randint(0, 10)
rgen.rvs(10)

array([9, 4, 1, 5, 9, 8, 0, 1, 8, 8])

In [31]:
# uniform : 실숫값

ugen = uniform(0, 1)
ugen.rvs(10)

array([0.44817869, 0.27774829, 0.15017226, 0.59109652, 0.78438653,
       0.8309138 , 0.02873519, 0.51901968, 0.38340142, 0.24554973])

In [32]:
# min_samples_leaf : 리프 노드가 되기 위한 최소 샘플의 개수
# 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [33]:
# 총 100번을 샘플링하여 교차 검증 수행 -> 최적의 하이퍼 파라미터를 찾음
# 교차 검증 수를 줄이면서 넓은 영역 효과적으로 탐색 가능


from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f09dfe02510>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f09dfe020d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f09dfe02850>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f09dfe02310>},
                   random_state=42)

In [34]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [35]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [36]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86
